#### The goal of the classifer is to assign scores based on how sure it is of an image having glasses or not. A score of 1 signifies a strong prediction of glasses, and 0 if there is a low chance of glasses. The latent vectors and scores are used to train a linear support vector machine. In the next google colab notebook.


In [4]:
!pip install Google-Colab-Transfer
import colab_transfer
colab_transfer.mount_google_drive()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


###Import Latent Vectors for faces from google Drive
#### Contains folder with latent vectors divided into glasses folder and no glasses folder


In [5]:
archive_name = 'faces_classifier.tar.gz'
colab_transfer.copy_file(file_name=archive_name,
                         destination=colab_transfer.get_path_to_home_of_local_machine())

File /content/faces_classifier.tar.gz already exists. Copy skipped.


In [ ]:
!du -sh $archive_name
!tar -xvf $archive_name

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib


In [9]:

data_dir2 = pathlib.Path("/content/GlassesClassifier")
print(data_dir2)

/content/GlassesClassifier


In [12]:
glasses = list(data_dir2.glob('Glasses/*'))
no_glasses = list(data_dir2.glob('NoGlasses/*'))
print(glasses[:5])
print(no_glasses[:5])

[PosixPath('/content/GlassesClassifier/Glasses/face-680.npy'), PosixPath('/content/GlassesClassifier/Glasses/face-974.npy'), PosixPath('/content/GlassesClassifier/Glasses/face-901.npy'), PosixPath('/content/GlassesClassifier/Glasses/face-275.npy'), PosixPath('/content/GlassesClassifier/Glasses/face-874.npy')]
[PosixPath('/content/GlassesClassifier/NoGlasses/face-940.npy'), PosixPath('/content/GlassesClassifier/NoGlasses/face-622.npy'), PosixPath('/content/GlassesClassifier/NoGlasses/face-804.npy'), PosixPath('/content/GlassesClassifier/NoGlasses/face-827.npy'), PosixPath('/content/GlassesClassifier/NoGlasses/face-328.npy')]


#### Load latent vector files into memory

In [ ]:
arr_glasses = []
arr_noglasses = []


for path in glasses:
  sample = np.load(path)
  arr_glasses.append(sample)

for idx,path in enumerate(no_glasses):
  #if idx != 621:          # 
  sample = np.load(path)
  arr_noglasses.append(sample)

np_glasses = np.array([x for x in arr_glasses])       # latent vectors with glasses
np_noglasses = np.array([x for x in arr_noglasses])   # latent vectors with no glasses


glasses_y = []
glasses_n = []

for i in range(np_glasses.shape[0]):
  glasses_y.append(1.0)                              # label 1.0 to represent glasses are present
for i in range(np_noglasses.shape[0]):
  glasses_n.append(0)                                # label 0 to represent no glasses

# print(np_glasses.shape)
# print(np_noglasses.shape)

In [159]:
combine = np.concatenate((np_glasses, np_noglasses), axis=0)      #combine no_glasses and glasses vectors together
combine_labels = np.concatenate((glasses_y, glasses_n), axis=0)   #combine the 0 and 1 labels together.
print(combine.shape)
print(combine_labels.shape)

(995, 18, 512)
(995,)


#### Split data into train, test and validation

In [160]:
from sklearn.model_selection import train_test_split
X = combine
y = combine_labels
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5)

nps_of_glasses = np_glasses.reshape(354, 9216).astype("float32")
nps_of_noglasses = np_noglasses.reshape(641, 9216).astype("float32")

In [161]:
print(x_train.shape) #latent vectors used to train
print(y_train.shape) #contains 1 or 0 depending on glasses or not
print(x_test.shape)  #test data
print(y_test.shape)
print(x_val.shape)   #validation data
print(y_val.shape)

(696, 18, 512)
(696,)
(149, 18, 512)
(149,)
(150, 18, 512)
(150,)


In [163]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#### Create Model

In [164]:
inputs = keras.Input(shape=(9216,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

#### Preprocess the data, reshape from (num_samples,18,512) to (num_samples,9216) aka (num_samples, 18 * 512)

In [168]:

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(696, 9216).astype("float32")
x_test = x_test.reshape(149, 9216).astype("float32") 
x_val = x_val.reshape(150,9216).astype("float32") 

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")
y_val = y_val.astype("float32")

In [169]:
print(x_train.shape)
print(y_train.shape)

(696, 9216)
(696,)
(1, 9216)


In [170]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=100,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

In [ ]:
history.history

In [173]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=32)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:10])
print(predictions)

Evaluate on test data
5/5 [==============================] - 0s 3ms/step - loss: 0.3516 - sparse_categorical_accuracy: 0.9664
test loss, test acc: [0.35161712765693665, 0.9664429426193237]
Generate predictions for 3 samples
[[1.0000000e+00 1.2785227e-14 7.3711829e-14 4.6938710e-14 2.9853121e-15
  3.6823226e-14 1.1294280e-16 2.7240043e-13 4.8443990e-16 1.5591987e-15]
 [1.2107247e-12 1.0000000e+00 6.9916538e-12 4.5262206e-16 3.8393195e-11
  6.0575386e-12 1.3206172e-11 5.8208104e-13 3.7082514e-11 1.1293619e-11]
 [1.0000000e+00 1.3128447e-18 4.8782005e-18 4.4712535e-16 1.5306852e-17
  2.5840724e-17 4.1089809e-20 1.5756398e-17 5.4391799e-19 7.0312539e-19]
 [9.9937516e-01 6.2344805e-04 5.4866251e-07 2.6292672e-09 6.6878968e-08
  2.5460631e-07 6.2738210e-09 2.0061385e-07 2.5111683e-08 2.5604683e-07]
 [1.0000000e+00 2.8107988e-18 2.5568610e-18 1.6920164e-15 4.0197021e-17
  1.6613529e-17 6.1586711e-20 4.2493596e-18 1.3240756e-18 1.3679001e-18]
 [1.0000000e+00 3.6385166e-12 1.0503666e-11 4.08226

In [174]:

combine = combine.reshape(995,9216).astype("float32")
#print(combine[-1].shape)
predictions = model.predict(combine[0:])

Assign scores from the predictions, to give to the linear SVM to find a boundary between glasses and no glasses

In [175]:

scores = []
for i in predictions:
  max_idx = np.argmax(i)  
  max_num = np.amax(i)
  if max_idx == 0:      # if its not glasses 
    score = 1 - max_num #inverse the score 
  else:                 # if it is glasses
    score = max_num     # assign the score
  scores.append(score)
print(np.array(scores).shape)

(995,)


#### Export the latent vectors and scores to google drive


In [176]:
import os 
dst_dir = "numpyarrglasses/"
os.makedirs(dst_dir, exist_ok=True)
filename = os.path.join(dst_dir, 'glasses_latent_vectors.npy')
np.save(filename,combine)

filename = os.path.join(dst_dir, 'scores.npy')
np.save(filename,scores)


In [177]:
folder_name = 'numpyarrglasses'
archive_name = 'numpyarrglasses.tar.gz'
!tar -cvf $archive_name $folder_name/
!du -sh $archive_name

numpyarrglasses/
numpyarrglasses/glasses_latent_vectors.npy
numpyarrglasses/scores.npy
numpyarrglasses/.ipynb_checkpoints/
35M	numpyarrglasses.tar.gz


In [178]:
colab_transfer.copy_file(file_name=archive_name,
                         source=colab_transfer.get_path_to_home_of_local_machine(),
                         destination=colab_transfer.get_path_to_home_of_google_drive())

File /content/drive/My Drive/numpyarrglasses.tar.gz already exists. Copy skipped.


# We can now proceed to pass the latent vectors and scores to a linear svm, to find the boundary.